In [1]:
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
import utm

In [7]:
kml_path = r"C:\Users\Alina\Desktop\SlyseL.kml"

# парсим
tree = ET.parse(kml_path)
root = tree.getroot()

kmlns = '{http://www.opengis.net/kml/2.2}'

# координаты линий
lines = []

for placemark in root.findall('.//' + kmlns + 'Placemark'):
    for linestring in placemark.findall('.//' + kmlns + 'LineString'):
        for coordinates in linestring.findall(kmlns + 'coordinates'):
    
            coord_list = [(float(lon), float(lat)) for lon, lat, _ in 
                          (point.split(',') for point in coordinates.text.strip().split(' '))]
            lines.append(coord_list)


lines_df_list = []
for i, line in enumerate(lines):
    
    temp_df = pd.DataFrame(line, columns=['Longitude', 'Latitude'])
    temp_df['Line'] = i + 1
    lines_df_list.append(temp_df)


lines_df = pd.concat(lines_df_list, ignore_index=True)

print(lines_df)


lines_df.to_csv('C:\\Users\\Alina\\Desktop\\lineswgs.txt', sep='\t', index=False)


    Longitude   Latitude  Line
0   88.018288  69.200532     1
1   88.019097  69.217267     1
2   88.023178  69.196987     2
3   88.024151  69.217244     2
4   88.028218  69.196942     3
5   88.029080  69.214793     3
6   88.033297  69.196850     4
7   88.034155  69.214812     4
8   88.038237  69.196855     5
9   88.039079  69.214774     5
10  88.043337  69.196806     6
11  88.044037  69.211113     6
12  88.048368  69.196871     7
13  88.049091  69.211089     7
14  88.053447  69.196779     8
15  88.054183  69.211019     8
16  88.058458  69.196801     9
17  88.059207  69.211062     9
18  88.063526  69.196688    10
19  88.064250  69.211016    10
20  87.991456  69.205997    11
21  88.119146  69.205292    11
22  87.986803  69.209676    12
23  88.118550  69.208862    12
24  87.996976  69.202446    13
25  88.118094  69.201687    13
26  88.002026  69.198845    14
27  88.118008  69.198142    14
28  87.981481  69.213295    15
29  88.118768  69.212470    15
30  88.012679  69.191523    16
31  88.1

In [11]:
from os import path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wellpathpy as wpp
import xarray as xr

In [6]:
from lxml import etree
import numpy as np

def read_local_kml(kml_path):
    with open(kml_path, 'rb') as kml_file:
        kml_data = kml_file.read()
    
    parser = etree.XMLParser(ns_clean=True)
    kml_tree = etree.fromstring(kml_data, parser=parser)
    nsmap = {'kml': 'http://www.opengis.net/kml/2.2'}
    placemarks = kml_tree.xpath('//kml:Placemark', namespaces=nsmap)
    for pm in placemarks:
        coord_lists = []
        coordinates = pm.xpath('.//kml:coordinates', namespaces=nsmap)
        for coord_elem in coordinates:
            coords_flat = np.fromstring(coord_elem.text, dtype=np.float64, sep=',')
            num_dims = 3 if ' ' in coord_elem.text else 2  # Простое предположение о размерности
            coords = coords_flat.reshape(-1, num_dims)
            print('Found a {0}D list (size: {1})'.format(num_dims, len(coords)))
            coord_lists.append(coords)
        msg = 'Total number of coordinate lists: {0} (size: {1})'
        print(msg.format(len(coord_lists), sum([len(coords) for coords in coord_lists])))

def main():
    # Укажите корректный путь к вашему локальному KML файлу
    kml_path = r"C:\Users\Alina\Desktop\SlyseL.kml"
    read_local_kml(kml_path)

if __name__ == '__main__':  # Правильная проверка __name__
    main()


Found a 3D list (size: 1)
Total number of coordinate lists: 1 (size: 1)
Found a 3D list (size: 1)
Total number of coordinate lists: 1 (size: 1)
Found a 3D list (size: 1)
Total number of coordinate lists: 1 (size: 1)
Found a 3D list (size: 1)
Total number of coordinate lists: 1 (size: 1)
Found a 3D list (size: 1)
Total number of coordinate lists: 1 (size: 1)
Found a 3D list (size: 1)
Total number of coordinate lists: 1 (size: 1)
Found a 3D list (size: 1)
Total number of coordinate lists: 1 (size: 1)
Found a 3D list (size: 1)
Total number of coordinate lists: 1 (size: 1)
Found a 3D list (size: 1)
Total number of coordinate lists: 1 (size: 1)
Found a 3D list (size: 1)
Total number of coordinate lists: 1 (size: 1)
Found a 3D list (size: 1)
Total number of coordinate lists: 1 (size: 1)
Found a 3D list (size: 1)
Total number of coordinate lists: 1 (size: 1)
Found a 3D list (size: 1)
Total number of coordinate lists: 1 (size: 1)
Found a 3D list (size: 1)
Total number of coordinate lists: 1 (s

C:\Users\Alina\AppData\Local\Temp\ipykernel_12868\3692089089.py:16: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  coords_flat = np.fromstring(coord_elem.text, dtype=np.float64, sep=',')


In [8]:
from lxml import etree
import pandas as pd
import numpy as np

def read_kml_to_df(kml_path):
   
    with open(kml_path, 'r', encoding='utf-8') as file:
        kml_content = file.read()

   
    root = etree.fromstring(kml_content.encode('utf-8'))

    
    ns = {'kml': "http://www.opengis.net/kml/2.2"}
    linestrings = root.xpath('//kml:LineString', namespaces=ns)

    
    lines_data = []

    for linestring in linestrings:

        for coords in linestring.xpath('.//kml:coordinates', namespaces=ns):
         
            coord_pairs = [tuple(map(float, coord.split(',')[:2])) for coord in coords.text.strip().split(' ')]

            lines_data.append(coord_pairs)


    df_list = [pd.DataFrame(line, columns=['Longitude', 'Latitude']) for line in lines_data]

    return df_list

def lines_to_numpy_arrays(df_list):
    return [line.to_numpy() for line in df_list]


kml_path = r"C:\Users\Alina\Desktop\SlyseL.kml"  
df_list = read_kml_to_df(kml_path) 
numpy_arrays = lines_to_numpy_arrays(df_list) 

for idx, arr in enumerate(numpy_arrays):
    print(f"Coordinates of line {idx+1} as NumPy array:\n{arr}\n")


Coordinates of line 1 as NumPy array:
[[88.01828783 69.2005325 ]
 [88.01909665 69.2172674 ]]

Coordinates of line 2 as NumPy array:
[[88.02317782 69.19698654]
 [88.02415144 69.217244  ]]

Coordinates of line 3 as NumPy array:
[[88.02821759 69.19694172]
 [88.02908036 69.21479271]]

Coordinates of line 4 as NumPy array:
[[88.03329665 69.19685046]
 [88.03415522 69.21481165]]

Coordinates of line 5 as NumPy array:
[[88.03823711 69.19685531]
 [88.0390791  69.21477384]]

Coordinates of line 6 as NumPy array:
[[88.04333676 69.19680637]
 [88.04403718 69.21111341]]

Coordinates of line 7 as NumPy array:
[[88.04836816 69.19687118]
 [88.04909051 69.21108929]]

Coordinates of line 8 as NumPy array:
[[88.05344712 69.19677934]
 [88.05418312 69.21101872]]

Coordinates of line 9 as NumPy array:
[[88.05845785 69.19680123]
 [88.05920749 69.2110619 ]]

Coordinates of line 10 as NumPy array:
[[88.0635264  69.19668778]
 [88.06425043 69.21101603]]

Coordinates of line 11 as NumPy array:
[[87.99145562 69.205